In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:

from solar_power.environment_prediction_model import SolarPredictionModelExtended 
from solar_power.solar_prediction_model import FeatureSolarPredictionModel
from electrical_load.load_prediction_model import BuildingEnergyModel

# * 모델 선언
# * 랜덤 시드 값을 모델 간 동일하게 42로 설정
# * 테스트 데이터 비율을 모델 간 동일하게 0.2로 설정

random_state = 42
test_size = 0.2

# * 기상 데이터 to 환경 감시 데이터 예측 모델
env_pred_model = SolarPredictionModelExtended("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
env_pred_model.load_data()
env_pred_model.hyperparameter_tuning()
env_pred_model.train_model()

# * 환경 감시 데이터 to 태양광 발전량 예측 모델
solar_pred_model = FeatureSolarPredictionModel("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
solar_pred_model.load_data()
solar_pred_model.feature_selection()
solar_pred_model.train_model()

# * 건물 전기 부하 예측 모델
energy_pred_model = BuildingEnergyModel("./electrical_load/merged_data.csv", random_state=random_state, test_size=test_size)
energy_pred_model.load_data()
energy_pred_model.train_model()

In [3]:
# * 태양광 발전량 예측 모델에 입력할 정규화된 입력 데이터
import numpy as np
rand_int = np.random.randint(0, len(env_pred_model.X_test))

test_x = env_pred_model.X_test.iloc[rand_int:rand_int+1, :]
test_x

,강수확률,일최저기온,하늘상태,일최고기온,습도,풍향,풍속,time
1318,0.3,0.5,1.0,0.285714,0.833333,0.085481,0.048387,0.978261


In [4]:
# * 예측한 태양광 발전량
res = env_pred_model.predict(test_x) # 기상 데이터 -> 환경감시 데이터
res = solar_pred_model.predict(res) # 환경감시 데이터 -> 태양광 발전량 데이터
res

array([8.99885641])

In [5]:
# * 실제 태양광 발전량
solar_pred_model.y_test.iloc[rand_int:rand_int+1, :]

,축구장,학생회관,중앙창고,학사과정,다산빌딩,시설관리동,대학C동,동물실험동,중앙도서관,LG도서관,신재생에너지동,삼성환경동,중앙연구기기센터,산업협력관,기숙사 B동
1318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# * 부하량 예측 모델에 입력할 정규화된 입력 데이터
test_x = energy_pred_model.X_test.iloc[rand_int:rand_int+1, :]
test_x

,time,1시간기온,1시간강수량,일최고기온,일최저기온,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
1318,22,25.0,0.0,29.0,21.0,False,False,False,False,False,False,True


In [7]:
# * 예측한 전기 부하량
res = energy_pred_model.predict(test_x)
res

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
0,164.4,456.3,430.2,483.0,138.4,280.7,35.7,54.3,0.0,0.0,...,42.0,41.9,40.6,30.9,30.5,29.7,56.4,22.2,70.2,18.6


In [8]:
# * 실제 전기 부하량
energy_pred_model.y_test.iloc[rand_int:rand_int+1, :]

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
1318,163.8,436.0,426.1,517.1,138.7,266.2,49.0,51.1,0.0,0.0,...,46.0,39.0,40.6,39.9,33.3,31.5,65.6,27.7,65.4,18.1


In [ ]:
# * 예측 발전량과 부하량을 통해 전기 요금 산출